# 🛠️ QEPC Environment Setup

This notebook is a **safe boot sequence** for QEPC.  
Use it when:

- You just opened Jupyter and want to make sure QEPC is wired correctly
- You changed environments (local vs cloud)
- You updated files / pulled from Git and want to confirm nothing is broken

It will:
1. Load the QEPC notebook context (paths, project root, imports)
2. Verify key data files exist
3. (Optional) Run full QEPC diagnostics


## 🧩 1. Load QEPC Notebook Context


In [4]:
# 1. Load QEPC Notebook Context (with auto path detection and project_root fallback)

import os
import sys
from pathlib import Path

# Try a normal import first
try:
    from notebook_context import *  # sets project_root, adds qepc/ to sys.path, prints banner (if defined that way)
    print("✅ Imported notebook_context directly.")
except ModuleNotFoundError:
    print("ℹ️ notebook_context not found on sys.path; trying to locate it...")

    cwd = Path.cwd()

    # Search a few candidate roots: current dir and parents
    candidate_roots = [
        cwd,
        cwd.parent,
        cwd.parent.parent,
    ]

    found_root = None
    for root in candidate_roots:
        if (root / "notebook_context.py").exists():
            found_root = root
            break

    if found_root is None:
        raise ModuleNotFoundError(
            f"Could not find notebook_context.py in {cwd} or its parents. "
            f"Make sure notebook_context.py is in your qepc_project root."
        )

    # Add that root to sys.path and switch working directory to it
    sys.path.insert(0, str(found_root))
    os.chdir(found_root)
    print(f"🔗 Added {found_root} to sys.path and changed working directory there.")

    # Now import again
    from notebook_context import *
    print("✅ Imported notebook_context after path adjustment.")

import pandas as pd

# 👇 Try to use project_root; if it's not defined, fall back to cwd
try:
    project_root  # just to see if it exists
except NameError:
    project_root = Path.cwd()
    print("⚠️ 'project_root' was not defined by notebook_context; "
          "using current working directory as project_root instead.")

print("Project root detected as:", project_root)
print("Current working directory:", os.getcwd())


✅ Imported notebook_context directly.
⚠️ 'project_root' was not defined by notebook_context; using current working directory as project_root instead.
Project root detected as: C:\Users\wdors\qepc_project
Current working directory: C:\Users\wdors\qepc_project


------

## 📂 2. Verify Project Structure & Key Data Files


In [5]:
# 2. Verify Project Structure & Key Data Files

from pathlib import Path

root = project_root

required_paths = {
    "Games.csv": root / "data" / "Games.csv",
    "TeamStatistics.csv": root / "data" / "raw" / "TeamStatistics.csv",
    "PlayerStatistics.csv": root / "data" / "raw" / "PlayerStatistics.csv",
    "Injury_Overrides (any)": [
        root / "data" / "Injury_Overrides.csv",
        root / "data" / "Injury_Overrides_data_driven.csv",
    ],
    "qepc_autoload.py": root / "qepc_autoload.py",
}

def check_path(name, path_or_list):
    if isinstance(path_or_list, list):
        exists_any = any(p.exists() for p in path_or_list)
        if exists_any:
            found = [str(p) for p in path_or_list if p.exists()]
            print(f"✅ {name}: found at {found}")
        else:
            print(f"⚠️ {name}: NOT FOUND in any expected location.")
    else:
        if path_or_list.exists():
            print(f"✅ {name}: {path_or_list}")
        else:
            print(f"⚠️ {name}: MISSING ({path_or_list})")

print("Checking key files...\n")
for name, path_obj in required_paths.items():
    check_path(name, path_obj)


Checking key files...

✅ Games.csv: C:\Users\wdors\qepc_project\data\Games.csv
✅ TeamStatistics.csv: C:\Users\wdors\qepc_project\data\raw\TeamStatistics.csv
✅ PlayerStatistics.csv: C:\Users\wdors\qepc_project\data\raw\PlayerStatistics.csv
✅ Injury_Overrides (any): found at ['C:\\Users\\wdors\\qepc_project\\data\\Injury_Overrides.csv', 'C:\\Users\\wdors\\qepc_project\\data\\Injury_Overrides_data_driven.csv']
✅ qepc_autoload.py: C:\Users\wdors\qepc_project\qepc_autoload.py


------

## 🔬 3. Quick QEPC Import Smoke Test


In [6]:
# 3. Quick QEPC Import Smoke Test

errors = []

def safe_import(label, fn):
    try:
        fn()
        print(f"✅ {label}: OK")
    except Exception as e:
        print(f"❌ {label}: ERROR -> {type(e).__name__}: {e}")
        errors.append((label, e))

def test_imports():
    import qepc_autoload as qa
    from qepc.core.lambda_engine import compute_lambda
    from qepc.core.simulator import run_qepc_simulation
    from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths
    from qepc.backtest.backtest_engine import run_season_backtest

safe_import("qepc core imports", test_imports)

if errors:
    print("\nSome imports failed. Check the messages above.")
else:
    print("\nAll core QEPC imports succeeded 🎉")


✅ qepc core imports: OK

All core QEPC imports succeeded 🎉


----

## 🧪 4. Full QEPC Diagnostics (Optional)


In [8]:
# 4. Full QEPC Diagnostics (Optional)

from qepc.utils.diagnostics import run_system_check

print("Running QEPC system diagnostics...\n")

diag = None

try:
    # First try passing project_root as a positional argument
    diag = run_system_check(project_root)
except TypeError:
    # If that fails, try with no arguments (function may handle paths internally)
    try:
        diag = run_system_check()
    except Exception as e:
        print("❌ run_system_check failed:", type(e).__name__, "->", e)
else:
    print("✅ Diagnostics completed without TypeError.")

# If the function returns something, show its type
if diag is not None:
    print("\nDiagnostics return object type:", type(diag))


Running QEPC system diagnostics...

🚀 QEPC SYSTEM DIAGNOSTICS INITIALIZED...

✅ Project Root: OK – Resolved to C:\Users\wdors\qepc_project

🔍 Checking required files...
✅ Canonical Schedule: OK – C:\Users\wdors\qepc_project\data\Games.csv
✅ Raw Player Stats: OK – C:\Users\wdors\qepc_project\data\raw\PlayerStatistics.csv
✅ Raw Team Stats: OK – C:\Users\wdors\qepc_project\data\raw\TeamStatistics.csv
✅ Autoload Context: OK – C:\Users\wdors\qepc_project\qepc_autoload.py
✅ Restore Guide (Notebook): OK – C:\Users\wdors\qepc_project\RESTORE_GUIDE.ipynb
✅ Restore Guide (Markdown): OK – C:\Users\wdors\qepc_project\notebooks\RESTORE_GUIDE.md


Check,Status,Details
Canonical Schedule,OK,C:\Users\wdors\qepc_project\data\Games.csv
Raw Player Stats,OK,C:\Users\wdors\qepc_project\data\raw\PlayerStatistics.csv
Raw Team Stats,OK,C:\Users\wdors\qepc_project\data\raw\TeamStatistics.csv
Autoload Context,OK,C:\Users\wdors\qepc_project\qepc_autoload.py
Restore Guide (Notebook),OK,C:\Users\wdors\qepc_project\RESTORE_GUIDE.ipynb
Restore Guide (Markdown),OK,C:\Users\wdors\qepc_project\notebooks\RESTORE_GUIDE.md



📊 Checking data schemas (where files exist)...
✅ Schema: data/Games.csv: OK – All expected columns present.
✅ Schema: data/Team_Stats.csv: OK – All expected columns present.
✅ Schema: data/raw/PlayerStatistics.csv: OK – All expected columns present.
✅ Schema: data/raw/TeamStatistics.csv: OK – All expected columns present.


Check,Status,Details
data/Games.csv,OK,All expected columns present.
data/Team_Stats.csv,OK,All expected columns present.
data/raw/PlayerStatistics.csv,OK,All expected columns present.
data/raw/TeamStatistics.csv,OK,All expected columns present.



🧪 Checking key module imports...
✅ Module: qepc.autoload.paths: OK – Loaded
✅ Module: qepc.core.lambda_engine: OK – Loaded
✅ Module: qepc.core.simulator: OK – Loaded
✅ Module: qepc.sports.nba.sim: OK – Loaded
✅ Module: qepc.sports.nba.strengths_v2: OK – Loaded
✅ Module: qepc.sports.nba.player_data: OK – Loaded
✅ Module: qepc.sports.nba.opponent_data: OK – Loaded
✅ Module: qepc.utils.backup: OK – Loaded
✅ Module: qepc.backtest.backtest_engine: OK – Loaded


Check,Status,Details
qepc.autoload.paths,OK,Loaded successfully
qepc.core.lambda_engine,OK,Loaded successfully
qepc.core.simulator,OK,Loaded successfully
qepc.sports.nba.sim,OK,Loaded successfully
qepc.sports.nba.strengths_v2,OK,Loaded successfully
qepc.sports.nba.player_data,OK,Loaded successfully
qepc.sports.nba.opponent_data,OK,Loaded successfully
qepc.utils.backup,OK,Loaded successfully
qepc.backtest.backtest_engine,OK,Loaded successfully



✨ DIAGNOSTICS COMPLETE.

Diagnostics return object type: <class 'dict'>


----

## ✅ 5. Ready Check


In [9]:
# 5. Ready Check & Notebook Launcher

if "errors" in globals() and errors:
    print("⚠️ QEPC environment has some issues. Check sections 2–4 above.")
else:
    print("✅ QEPC environment looks good.\n")
    print("Next steps:")
    print("  • 🧪 Open: notebooks/qepc_sandbox.ipynb  (daily experiments, today’s games)")
    print("  • 📊 Open: notebooks/qepc_backtest.ipynb (model performance & backtests)")


✅ QEPC environment looks good.

Next steps:
  • 🧪 Open: notebooks/qepc_sandbox.ipynb  (daily experiments, today’s games)
  • 📊 Open: notebooks/qepc_backtest.ipynb (model performance & backtests)


----